In [ ]:
CREATE TABLE IF NOT EXISTS FACT_EVENTS (
    ID_EVENT         NUMBER(38,0),
    ID_GROUP         NUMBER(38,0),
    ID_VENUE         NUMBER(38,0),
    CREATED          TIMESTAMP_NTZ,
    DESCRIPTION      VARCHAR,
    DURATION         NUMBER(38,0),
    EVENT_URL        VARCHAR,
    FEE_ACCEPTS      VARCHAR,
    FEE_AMOUNT       NUMBER(38,0),     -- si quieres decimales, cambia a NUMBER(38,2)
    FEE_CURRENCY     VARCHAR,
    FEE_DESCRIPTION  VARCHAR,
    FEE_LABEL        VARCHAR,
    FEE_REQUIRED     NUMBER(38,0),
    HEADCOUNT        NUMBER(38,0),
    HOW_TO_FIND_US   VARCHAR,
    MAYBE_RSVP_COUNT NUMBER(38,0),
    EVENT_NAME       VARCHAR,
    PHOTO_URL        VARCHAR,
    RATING_AVERAGE   NUMBER(38,0),     -- si prefieres decimales, usa NUMBER(38,2)
    RATING_COUNT     NUMBER(38,0),
    RSVP_LIMIT       NUMBER(38,0),
    EVENT_STATUS     VARCHAR,
    EVENT_TIME       TIMESTAMP_NTZ,
    UPDATED          TIMESTAMP_NTZ,
    UTC_OFFSET       NUMBER(38,0),
    VISIBILITY       VARCHAR,
    WAITLIST_COUNT   NUMBER(38,0),
    WHY              VARCHAR,
    YES_RSVP_COUNT   NUMBER(38,0),
    PRIMARY KEY (ID_EVENT)             -- declarativo
);

MERGE INTO DATA_SLV.FACT_EVENTS AS d
USING (
  SELECT DISTINCT
    TRY_TO_NUMBER(e.event_id)                       AS id_event,
    TRY_TO_NUMBER(e.group_id)                       AS id_group,
    TRY_TO_NUMBER(e.venue_id)                       AS id_venue,
    e.created                                       AS created,
    NULLIF(TRIM(e.description), '')                 AS description,
    TRY_TO_NUMBER(e.duration)                       AS duration,
    NULLIF(TRIM(e.event_url), '')                   AS event_url,
    NULLIF(TRIM(e."fee.accepts"), '')               AS fee_accepts,
    -- Tu destino es NUMBER(38,0). Si el origen trae decimales, redondeo:
    TRY_TO_NUMBER(ROUND(e."fee.amount", 0))         AS fee_amount,
    NULLIF(TRIM(e."fee.currency"), '')              AS fee_currency,
    NULLIF(TRIM(e."fee.description"), '')           AS fee_description,
    NULLIF(TRIM(e."fee.label"), '')                 AS fee_label,
    TRY_TO_NUMBER(e."fee.required")                 AS fee_required,
    TRY_TO_NUMBER(e.headcount)                      AS headcount,
    NULLIF(TRIM(e.how_to_find_us), '')              AS how_to_find_us,
    TRY_TO_NUMBER(e.maybe_rsvp_count)               AS maybe_rsvp_count,
    NULLIF(TRIM(e.event_name), '')                  AS event_name,
    NULLIF(TRIM(e.photo_url), '')                   AS photo_url,
    TRY_TO_NUMBER(ROUND(e."rating.average", 0))     AS rating_average,
    TRY_TO_NUMBER(e."rating.count")                 AS rating_count,
    TRY_TO_NUMBER(e.rsvp_limit)                     AS rsvp_limit,
    NULLIF(TRIM(e.event_status), '')                AS event_status,
    e.event_time                                    AS event_time,
    e.updated                                       AS updated,
    TRY_TO_NUMBER(e.utc_offset)                     AS utc_offset,
    NULLIF(TRIM(e.visibility), '')                  AS visibility,
    TRY_TO_NUMBER(e.waitlist_count)                 AS waitlist_count,
    NULLIF(TRIM(e.why), '')                         AS why,
    TRY_TO_NUMBER(e.yes_rsvp_count)                 AS yes_rsvp_count
  FROM MEETUP_DB.DATA_BRZ.EVENTS AS e
  WHERE TRY_TO_NUMBER(e.event_id) IS NOT NULL
) AS s
ON d.ID_EVENT = s.id_event

WHEN MATCHED THEN UPDATE SET
  d.ID_GROUP         = s.id_group,
  d.ID_VENUE         = s.id_venue,
  d.CREATED          = s.created,
  d.DESCRIPTION      = s.description,
  d.DURATION         = s.duration,
  d.EVENT_URL        = s.event_url,
  d.FEE_ACCEPTS      = s.fee_accepts,
  d.FEE_AMOUNT       = s.fee_amount,
  d.FEE_CURRENCY     = s.fee_currency,
  d.FEE_DESCRIPTION  = s.fee_description,
  d.FEE_LABEL        = s.fee_label,
  d.FEE_REQUIRED     = s.fee_required,
  d.HEADCOUNT        = s.headcount,
  d.HOW_TO_FIND_US   = s.how_to_find_us,
  d.MAYBE_RSVP_COUNT = s.maybe_rsvp_count,
  d.EVENT_NAME       = s.event_name,
  d.PHOTO_URL        = s.photo_url,
  d.RATING_AVERAGE   = s.rating_average,
  d.RATING_COUNT     = s.rating_count,
  d.RSVP_LIMIT       = s.rsvp_limit,
  d.EVENT_STATUS     = s.event_status,
  d.EVENT_TIME       = s.event_time,
  d.UPDATED          = s.updated,
  d.UTC_OFFSET       = s.utc_offset,
  d.VISIBILITY       = s.visibility,
  d.WAITLIST_COUNT   = s.waitlist_count,
  d.WHY              = s.why,
  d.YES_RSVP_COUNT   = s.yes_rsvp_count

WHEN NOT MATCHED THEN INSERT (
  ID_EVENT, ID_GROUP, ID_VENUE, CREATED, DESCRIPTION, DURATION, EVENT_URL,
  FEE_ACCEPTS, FEE_AMOUNT, FEE_CURRENCY, FEE_DESCRIPTION, FEE_LABEL, FEE_REQUIRED,
  HEADCOUNT, HOW_TO_FIND_US, MAYBE_RSVP_COUNT, EVENT_NAME, PHOTO_URL,
  RATING_AVERAGE, RATING_COUNT, RSVP_LIMIT, EVENT_STATUS, EVENT_TIME, UPDATED,
  UTC_OFFSET, VISIBILITY, WAITLIST_COUNT, WHY, YES_RSVP_COUNT
) VALUES (
  s.id_event, s.id_group, s.id_venue, s.created, s.description, s.duration, s.event_url,
  s.fee_accepts, s.fee_amount, s.fee_currency, s.fee_description, s.fee_label, s.fee_required,
  s.headcount, s.how_to_find_us, s.maybe_rsvp_count, s.event_name, s.photo_url,
  s.rating_average, s.rating_count, s.rsvp_limit, s.event_status, s.event_time, s.updated,
  s.utc_offset, s.visibility, s.waitlist_count, s.why, s.yes_rsvp_count
);